In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
os.environ["CUDA_VISIBLE_DEVICES"]="3" #,1,2,3

In [2]:
import os 
import torch
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from os.path import join as pjoin
from torch.distributions import Categorical
import json
import clip

import options.option_transformer as option_trans
import models.vqvae as vqvae
import utils.utils_model as utils_model
import utils.eval_trans as eval_trans
from dataset import dataset_TM_train
from dataset import dataset_TM_eval
from dataset import dataset_tokenize
import models.t2m_trans_uplow as trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from exit.utils import get_model, visualize_2motions, generate_src_mask, uniform, cosine_schedule
from einops import rearrange, repeat
import os
import torch.nn.functional as F
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
# os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6,7"

In [3]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 't2m'
args.down_t = 2

# train_loader_token = dataset_tokenize.DATALoader(args.dataname, 1, unit_length=2**args.down_t)

from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

dataset_opt_path = 'checkpoints/kit/Comp_v6_KLD005/opt.txt' if args.dataname == 'kit' else 'checkpoints/t2m/Comp_v6_KLD005/opt.txt'

wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt


  0%|          | 0/1460 [00:00<?, ?it/s]

100%|██████████| 1460/1460 [00:00<00:00, 2759.29it/s]


Pointer Pointing at 0
Reading checkpoints/t2m/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 28) Completed!!


In [4]:
##### ---- Network ---- #####
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

# https://github.com/openai/CLIP/issues/111
class TextCLIP(torch.nn.Module):
    def __init__(self, model) :
        super(TextCLIP, self).__init__()
        self.model = model
        
    def forward(self,text):
        return self.model.encode_text(text)
clip_model = TextCLIP(clip_model)

In [5]:
args.nb_code = 8192 # 512 # 
args.code_dim = 32 # 512 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None
args.quantizer = 'ema_reset'
args.mu = 0.99
args.clip_dim = 512
args.block_size = 51
args.n_head_gpt = 16
args.drop_out_rate = 0.1
args.ff_rate = 4


######## LARGE ########
args.num_layers = 9
args.embed_dim_gpt = 1024
args.resume_trans = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/2023-08-04-23-08-30_HML3D_36_token1stStage_cdim8192_32_lr0.0001_mask.5-1/net_last.pth'
args.resume_pth = f'/home/epinyoan/git/MaskText2Motion/T2M-BD/output/vq/2023-08-08-00-29-40_16_VQVAE_upperlower_notShareCB_20batchResetNRandom_8192x32/net_last.pth'

if args.dataset_name == 'kit':
    args.vq_name = 'VQVAE'
elif args.dataset_name == 't2m':
    args.vq_name = '2023-08-08-00-29-40_16_VQVAE_upperlower_notShareCB_20batchResetNRandom_8192x32'


args.exp_name = 'TEMP'
args.out_dir = 'output_GPT_Final'
args.out_dir = os.path.join(args.out_dir, f'{args.exp_name}')
args.vq_dir= f'output/vq/{args.vq_name}'
codebook_dir = f'{args.vq_dir}/codebook/'
os.makedirs(args.out_dir, exist_ok = True)
os.makedirs(args.vq_dir, exist_ok = True)
os.makedirs(codebook_dir, exist_ok = True)

from models.vqvae_sep import VQVAE_SEP
net = VQVAE_SEP(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate)
class VQVAE_WRAPPER(torch.nn.Module):
    def __init__(self, vqvae) :
        super().__init__()
        self.vqvae = vqvae
        
    def forward(self, *args, **kwargs):
        return self.vqvae(*args, **kwargs)
ckpt = torch.load(args.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['net'], strict=True)
# net = torch.nn.DataParallel(net)
net = VQVAE_WRAPPER(net)
    
trans_encoder = trans.Text2Motion_Transformer(net, 
                                    num_vq=args.nb_code, 
                                    embed_dim=args.embed_dim_gpt, 
                                    clip_dim=args.clip_dim, 
                                    block_size=args.block_size, 
                                    num_layers=args.num_layers, 
                                    n_head=args.n_head_gpt, 
                                    drop_out_rate=args.drop_out_rate, 
                                    fc_rate=args.ff_rate)



ckpt = torch.load(args.resume_trans, map_location='cpu')
# trans_encoder.load_state_dict(ckpt['trans'], strict=True)
trans_encoder = torch.nn.DataParallel(trans_encoder)
net.eval()
net.cuda()
trans_encoder.train()
trans_encoder.cuda()
''

''

In [6]:
from tqdm import tqdm
if len(os.listdir(codebook_dir)) == 0:
    train_loader_token = dataset_tokenize.DATALoader(args.dataname, 1, unit_length=2**args.down_t)
    for batch in tqdm(train_loader_token):
        pose, name = batch
        bs, seq = pose.shape[0], pose.shape[1]

        pose = pose.cuda().float() # bs, nb_joints, joints_dim, seq_len
        target = net(pose, type='encode')
        target = target.cpu().numpy()
        np.save(pjoin(codebook_dir, name[0] +'.npy'), target)

In [7]:
args.batch_size = 2
num_workers = 8
train_loader = dataset_TM_train.DATALoader(args.dataname, args.batch_size, args.nb_code, codebook_dir, unit_length=2**args.down_t, num_workers=num_workers, up_low_sep=True)
train_loader_iter = dataset_TM_train.cycle(train_loader)

  0%|          | 0/23384 [00:00<?, ?it/s]

100%|██████████| 23384/23384 [00:06<00:00, 3550.26it/s]


In [8]:
args.pkeep = .5
batch = next(iter(train_loader))
clip_text, target, m_tokens_len = batch

In [9]:
batch_size, max_len = target.shape[:2]
target, m_tokens_len = target.cuda(), m_tokens_len.cuda()
bs = target.shape[0]

text = clip.tokenize(clip_text, truncate=True).cuda()

feat_clip_text = clip_model(text).float()
target_upper = target[..., 0]
target_lower = target[..., 1]

# [INFO] Swap input index
if args.pkeep == -1:
    proba = np.random.rand(1)[0]
    mask = torch.bernoulli(proba * torch.ones(target_upper.shape,
                                                        device=target.device))
else:
    mask = torch.bernoulli(args.pkeep * torch.ones(target_upper.shape,
                                                        device=target.device))
# random only motion token (not pad token). To prevent pad token got mixed up.
seq_mask_no_end = generate_src_mask(max_len, m_tokens_len)
mask = torch.logical_or(mask, ~seq_mask_no_end).int()
r_indices = torch.randint_like(target_upper, args.nb_code)
input_indices = mask*target_upper+(1-mask)*r_indices

In [10]:
import random
random.randint(0, 1)

0

In [11]:
import random
proba = random.randint(0, 2)/10
mask_lower = torch.bernoulli(proba * torch.ones(target_lower.shape,
                                                        device=target.device))
mask_lower = torch.logical_or(mask_lower, ~seq_mask_no_end).int()
r_indices_lower = torch.randint_like(target_lower, args.nb_code)
input_indices_lower = mask_lower*target_upper+(1-mask_lower)*r_indices_lower

In [12]:
# Time step masking
mask_id = get_model(net).vqvae.num_code + 2
# rand_time = uniform((batch_size,), device = target.device)
# rand_mask_probs = cosine_schedule(rand_time)
rand_mask_probs = torch.zeros(batch_size, device = m_tokens_len.device).float().uniform_(0.5, 1)
num_token_masked = (m_tokens_len * rand_mask_probs).round().clamp(min = 1)
seq_mask = generate_src_mask(max_len, m_tokens_len+1)
batch_randperm = torch.rand((batch_size, max_len), device = target.device) - seq_mask_no_end.int()
batch_randperm = batch_randperm.argsort(dim = -1)
mask_token = batch_randperm < rearrange(num_token_masked, 'b -> b 1')

# masked_target = torch.where(mask_token, input=input_indices, other=-1)
masked_input_indices = torch.where(mask_token, mask_id, input_indices)



In [13]:
att_txt = None # CFG: torch.rand((seq_mask.shape[0], 1)) > 0.1
cls_pred = trans_encoder(masked_input_indices, input_indices_lower, feat_clip_text, src_mask = seq_mask, att_txt=att_txt)[:, 1:]

In [14]:
# [INFO] Compute xent loss as a batch
weights = seq_mask_no_end / (seq_mask_no_end.sum(-1).unsqueeze(-1) * seq_mask_no_end.shape[0])
cls_pred_seq_masked = cls_pred[seq_mask_no_end, :].view(-1, cls_pred.shape[-1])
target_seq_masked = target_upper[seq_mask_no_end]
weight_seq_masked = weights[seq_mask_no_end]
loss_cls = F.cross_entropy(cls_pred_seq_masked, target_seq_masked, reduction = 'none')
loss_cls = (loss_cls * weight_seq_masked).sum()

### Eval

In [15]:
val_loader = dataset_TM_eval.DATALoader(args.dataname, True, 32, w_vectorizer)

  5%|▍         | 201/4384 [00:00<00:02, 2000.56it/s]

100%|██████████| 4384/4384 [00:02<00:00, 1938.24it/s]

Pointer Pointing at 0


In [16]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
    def add_video(self, *args):
        print(*args)
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')

test
./Test/FID fid nb_iter


In [17]:
########## Test Single Sample ##########
# word_embeddings, pos_one_hots, clip_text, sent_len, pose, m_length, token, name = next(iter(val_loader))
# seq = pose.shape[1]
# m_length = m_length.cuda()
m_length = torch.tensor([180, 54]).cuda() # length: [196, 54]
m_tokens_len = torch.ceil((m_length)/4)
feat_clip_text = torch.rand([2, 512]) # [2, 512]
feat_clip_text = feat_clip_text.cuda()

target_lower = torch.randint(0, 8192, [2, 50])
seq_mask_no_end = generate_src_mask(max_len, m_tokens_len)
target_lower[~seq_mask_no_end] = 8193
target_lower = target_lower.cuda()
target_lower.scatter_(dim=-1, index=m_tokens_len[:, None].long(), value=8192)

import timeit
total = 0
num_all = 0
blank_id = get_model(trans_encoder).num_vq
for k in range(1):
    start = timeit.default_timer()
    index_motion, temp = trans_encoder(feat_clip_text, target_lower, type="sample", m_length=m_length, if_test=True, CFG=-1)
    index_motion = torch.cat((index_motion[..., None], target_lower[..., None]), dim=-1)
    pred_pose = net(index_motion[k:k+1, :int(m_tokens_len[k].item())], type='decode')

    stop = timeit.default_timer()
    total += (stop - start)
    num_all += 1
total/num_all


0.4808988790027797

In [ ]:
pred_pose_eval, pose, m_length, clip_text, best_fid, best_iter, best_div, best_top1, best_top2, best_top3, best_matching, best_multi, writer, logger = \
    eval_trans.evaluation_transformer_uplow(args.out_dir, 
        val_loader, 
        net, 
        trans_encoder, 
        logger, 
        writer, 
        0, 
        best_fid=1000, 
        best_iter=0, 
        best_div=100, 
        best_top1=0, 
        best_top2=0, 
        best_top3=0, 
        best_matching=100, 
        clip_model=clip_model, 
        dataname=args.dataname,
        eval_wrapper=eval_wrapper,
        num_repeat=1,
        rand_pos=True,
        CFG=-1)